# Part A - Baseline model 

Build a baseline model (5 marks) 

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

Submit your Jupyter Notebook with your code and comments.

In [1]:
#import libraries
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Input

from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter('ignore', FutureWarning)

2025-01-21 18:37:51.695796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737481071.719729  680176 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737481071.726849  680176 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 18:37:51.750439: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Import and check data

In [2]:
# download the data and read it into a pandas dataframe.
concrete_data = pd.read_csv("concrete_data.csv")

concrete_data.shape

(1030, 9)

In [3]:
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.tail()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77
1029,260.9,100.5,78.3,200.6,8.6,864.5,761.5,28,32.40


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The file looks ready to be used :)

#### Split data into predictors and target

In [6]:
concrete_data.columns

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [7]:
predictors = concrete_data[concrete_data.columns[concrete_data.columns != 'Strength']] 
target = concrete_data['Strength'] 
print(f"predictors shape: {predictors.shape}")
print(f"target shape: {target.shape}")

predictors shape: (1030, 8)
target shape: (1030,)


### Build the neural network

In [8]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(8,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')   
    return model

### Train and test the model

In [9]:
# build the model
model = regression_model()

2025-01-21 18:37:55.061642: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
# inspect the model
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │            90 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101 (404.00 B)

 Trainable params: 101 (404.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [11]:
#i) Split the data randomly
x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state = 9)

In [12]:
print(f"x_train.shape: {x_train.shape}, x_test.shape: {x_test.shape}, y_train.shape: {y_train.shape}, y_test.shape: {y_test.shape}")

x_train.shape: (721, 8), x_test.shape: (309, 8), y_train.shape: (721,), y_test.shape: (309,)


In [13]:
%%time
#ii) fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=2)

Epoch 1/50
23/23 - 1s - 42ms/step - loss: 25100.3281 - val_loss: 12252.2383
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 6078.7461 - val_loss: 2152.0583
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 1163.8477 - val_loss: 809.8466
Epoch 4/50
23/23 - 0s - 5ms/step - loss: 770.9557 - val_loss: 784.9658
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 745.4727 - val_loss: 758.5526
Epoch 6/50
23/23 - 0s - 5ms/step - loss: 718.1478 - val_loss: 737.4070
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 692.3880 - val_loss: 714.9540
Epoch 8/50
23/23 - 0s - 5ms/step - loss: 666.7967 - val_loss: 692.5121
Epoch 9/50
23/23 - 0s - 5ms/step - loss: 640.8186 - val_loss: 671.7390
Epoch 10/50
23/23 - 0s - 5ms/step - loss: 616.3515 - val_loss: 650.0067
Epoch 11/50
23/23 - 0s - 5ms/step - loss: 592.1005 - val_loss: 628.9905
Epoch 12/50
23/23 - 0s - 5ms/step - loss: 568.2179 - val_loss: 607.9261
Epoch 13/50
23/23 - 0s - 5ms/step - loss: 545.3815 - val_loss: 588.2196
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 523.0853 - val_loss: 56

In [14]:
#iii) evaluate the model
mse = model.evaluate(x_test, y_test, verbose=2)
print(f"MSE: {mse}")

10/10 - 0s - 8ms/step - loss: 199.3756
MSE: 199.3756103515625


### Repeat steps i) to iii) 50 times and show mean and standard deviation of the results

In [15]:
model = regression_model()
mean_squared_errors = []

In [16]:
%%time

for i in range(50):
    # i) split data
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=9)
    ## ii) fit model
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=0)
    # iii) evaluate model
    mse = model.evaluate(x_test, y_test, verbose=0)
    mean_squared_errors.append(mse)

CPU times: user 6min 7s, sys: 20.7 s, total: 6min 28s
Wall time: 5min 27s


Calculate mean and std. deviation

In [17]:
#Calculate the mean and standard deviation of the mean squared errors
mse_mean = np.mean(mean_squared_errors)
mse_std = np.std(mean_squared_errors)

print(f"Mean of calculated mse: {mse_mean}")
print(f"Standard deviation of calculated mse: {mse_std}")

Mean of calculated mse: 77.25257614135742
Standard deviation of calculated mse: 72.64313537076397


**Executions:**

First execution:<br>
Mean of calculated mse: 66.65317337036133 <br>
Std. deviation of calculated mse: 64.99147919799374 <br>
CPU times: user 6min 11s, sys: 19.4 s, total: 6min 31s
Wall time: 5min 33s
<br>
<br>
Second execution:<br>
Mean of calculated mse:  77.25257614135742  <br>
Std. deviation of calculated mse:  72.64313537076397 <br>
CPU times: user 6min 7s, sys: 20.7 s, total: 6min 28s
Wall time: 5min 27s